# Introducción para la optimización en la toma de decisiones
- Alumna: Rosa Huamaní Pucho - 20200422

## Ejercicio 1: el problema de la dieta
---


In [1]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

### Paso 1: inciciamos el modelo

In [2]:
import pulp as pp

In [3]:
model = pp.LpProblem(name='dieta-problem', # just the name
                     sense=pp.LpMinimize) # type of problem

### Paso 2: declarar las VARIABLES

In [4]:
# cuanto de VegaVita?
VegaVita = pp.LpVariable(name="VegaVita",  
                    lowBound=0,  
                    cat='Integer')

# cuanto de HappyHealth?
HappyHealth = pp.LpVariable(name="HappyHealth",
                 lowBound=0,
                 cat='Integer')

### Paso 3: creamos la función para optimizar

In [5]:
VegaVitaCoeff=0.2
HappyHealthCoeff=0.3
obj_func = VegaVitaCoeff*VegaVita + HappyHealthCoeff*HappyHealth

### Paso 4: representamos las restricciones

In [6]:
C1= pp.LpConstraint(name='vitaminac Constraint',   # just the name
                    e= 20*VegaVita + 30*HappyHealth, rhs=60, # linear combination of constraint and rhs
                    sense=pp.LpConstraintGE) 
C2= pp.LpConstraint(name='calcio Constraint',   
                    e= 500*VegaVita + 250*HappyHealth, rhs=1000, 
                    sense=pp.LpConstraintGE) 
C3= pp.LpConstraint(name='hierro Constraint',   
                    e= 9*VegaVita + 2*HappyHealth, rhs=18, 
                    sense=pp.LpConstraintGE) 
C4= pp.LpConstraint(name='niacina Constraint',   
                    e= 2*VegaVita + 10*HappyHealth, rhs=20, 
                    sense=pp.LpConstraintGE)
C5= pp.LpConstraint(name='magnesio Constraint',   
                    e= 60*VegaVita + 90*HappyHealth, rhs=360, 
                    sense=pp.LpConstraintGE)  

### Paso 5: Construyendo el modelo

In [7]:
model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

### Paso 6: resolviendo el modelo

In [8]:
solver_list = pp.listSolvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD']


In [9]:
#solverToUse = pp.COIN_CMD(msg=False)
model.solve(0);

In [10]:
import pandas as pd

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Model Status': 'Optimal',
 'Optimal Solution': 1.2000000000000002,
 'HappyHealth': 2.0,
 'VegaVita': 3.0}

In [11]:
#or
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

,Optimal Solution,HappyHealth,VegaVita
Model Status,,,
Optimal,1.2000000000000002,2.0,3.0


**Respuesta** : Para satisfacer los requerimientos nutricionales mínimos y gastar lo menos posible, el modelo sugiere adquirir 2 tabletas de HappyHealth y 3 tabletas de VegaVita, lo cual costaría $1.2 en total.

## Ejercicio 2: minimizar el número de conductores
---

In [12]:
# Paso 1: iniciamos el modelo 2
model2 = pp.LpProblem(name='horario-problem', 
                     sense=pp.LpMinimize) 

In [13]:
# ´Paso 2: declaramos las variables
DR_0_8 = pp.LpVariable(name="DR_0_8", lowBound=0, cat="Integer")
DR_4_12 = pp.LpVariable(name="DR_4_12", lowBound=0, cat="Integer")
DR_8_16 = pp.LpVariable(name="DR_8_16", lowBound=0, cat="Integer")
DR_12_20 = pp.LpVariable(name="DR_12_20", lowBound=0, cat="Integer")
DR_16_0 = pp.LpVariable(name="DR_16_0", lowBound=0, cat="Integer")
DR_20_4 = pp.LpVariable(name="DR_20_4", lowBound=0, cat="Integer")

In [14]:
# Paso 3: creamos la función para optimizar
model2 += (DR_0_8 + DR_4_12 + DR_8_16 + DR_12_20 + DR_16_0 + DR_20_4), "Total Drivers"

In [15]:
# Paso 4: presentamos las restricciones
C1 = pp.LpConstraint(name="Shift_0_4",
                     e= DR_0_8 + DR_20_4,  
                     rhs=4,               
                     sense=pp.LpConstraintGE)

C2 = pp.LpConstraint(name="Shift_4_8",
                     e= DR_0_8 + DR_4_12,
                     rhs=8,
                     sense=pp.LpConstraintGE)

C3 = pp.LpConstraint(name="Shift_8_12",
                     e= DR_4_12 + DR_8_16,
                     rhs=10,
                     sense=pp.LpConstraintGE)

C4 = pp.LpConstraint(name="Shift_12_16",
                     e= DR_8_16 + DR_12_20,
                     rhs=7,
                     sense=pp.LpConstraintGE)

C5 = pp.LpConstraint(name="Shift_16_20",
                     e= DR_12_20 + DR_16_0,
                     rhs=12,
                     sense=pp.LpConstraintGE)

C6 = pp.LpConstraint(name="Shift_20_0",
                     e= DR_16_0 + DR_20_4,
                     rhs=4,
                     sense=pp.LpConstraintGE)

In [16]:
# Paso 5: construyendo el modelo
model2 += C1
model2 += C2
model2 += C3
model2 += C4
model2 += C5
model2 += C6

In [17]:
# Paso 6: resolviendo el modelo
solver_list = pp.listSolvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD']


In [18]:
model2.solve(pp.PULP_CBC_CMD(msg=False))

1

In [19]:
print("Status:", pp.LpStatus[model2.status])
print("Total Drivers Needed:", pp.value(model2.objective))
print("Drivers starting at 0-8:", pp.value(DR_0_8))
print("Drivers starting at 4-12:", pp.value(DR_4_12))
print("Drivers starting at 8-16:", pp.value(DR_8_16))
print("Drivers starting at 12-20:", pp.value(DR_12_20))
print("Drivers starting at 16-0:", pp.value(DR_16_0))
print("Drivers starting at 20-4:", pp.value(DR_20_4))

Status: Optimal
Total Drivers Needed: 26.0
Drivers starting at 0-8: 0.0
Drivers starting at 4-12: 10.0
Drivers starting at 8-16: 0.0
Drivers starting at 12-20: 12.0
Drivers starting at 16-0: 0.0
Drivers starting at 20-4: 4.0


## Ejercicio 3: Toma de decisiones con criterios múltiples

### Paso 1: preparar la data para las comparaciones

In [20]:
%%html

<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQbKRIfZ6h_PrSW74V9F_7kIUbp0rfzexw-66DihXQuF_P5JwxLXIWwDLy3Oiz_TZ1XF6F59C2eunTj/pubhtml" width="600" height="300" ></iframe>

In [21]:
# the link to the data

linkGoogle= 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQbKRIfZ6h_PrSW74V9F_7kIUbp0rfzexw-66DihXQuF_P5JwxLXIWwDLy3Oiz_TZ1XF6F59C2eunTj/pub?output=xlsx'# the link to the data

In [22]:
!pip install openpyxl


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Paso 2: abrir cada hoja de excel

In [23]:
# opening the comparissons

import pandas as pd

pairwise_inclusion=pd.read_excel(linkGoogle,sheet_name='inclusión social', index_col=0)
pairwise_beneficiarios=pd.read_excel(linkGoogle,sheet_name='beneficiarios', index_col=0)
pairwise_premios=pd.read_excel(linkGoogle,sheet_name='premios', index_col=0)
pairwise_criterios=pd.read_excel(linkGoogle,sheet_name='criterios', index_col=0)

In [24]:
pairwise_criterios

,INCLUSION,BENEFICIARIOS,RECONOCIMIENTO
INCLUSION,1.0,0.33,0.20
BENEFICIARIOS,0.2,1.00,0.25
RECONOCIMIENTO,5.0,4.00,1.00


### Paso 3: Transforme todas las matrices en comparaciones por pares

In [25]:
import networkx as nx

G_inclusion = nx.from_pandas_adjacency(pairwise_inclusion,create_using=nx.MultiDiGraph())

# pairwise
G_inclusion.edges(data=True)

OutMultiEdgeDataView([('PRONABEC', 'PRONABEC', {'weight': 1.0}), ('PRONABEC', 'QALIWARMA', {'weight': 7.0}), ('PRONABEC', 'JUNTOS', {'weight': 0.25}), ('PRONABEC', 'FONCODES', {'weight': 3.0}), ('PRONABEC', 'CUNAMAS', {'weight': 5.0}), ('QALIWARMA', 'QALIWARMA', {'weight': 1.0}), ('QALIWARMA', 'PRONABEC', {'weight': 0.14}), ('QALIWARMA', 'JUNTOS', {'weight': 0.16}), ('QALIWARMA', 'FONCODES', {'weight': 0.5}), ('QALIWARMA', 'CUNAMAS', {'weight': 3.0}), ('JUNTOS', 'JUNTOS', {'weight': 1.0}), ('JUNTOS', 'PRONABEC', {'weight': 4.0}), ('JUNTOS', 'QALIWARMA', {'weight': 6.0}), ('JUNTOS', 'FONCODES', {'weight': 5.0}), ('JUNTOS', 'CUNAMAS', {'weight': 7.0}), ('FONCODES', 'FONCODES', {'weight': 1.0}), ('FONCODES', 'PRONABEC', {'weight': 0.33}), ('FONCODES', 'QALIWARMA', {'weight': 2.0}), ('FONCODES', 'JUNTOS', {'weight': 0.2}), ('FONCODES', 'CUNAMAS', {'weight': 5.0}), ('CUNAMAS', 'CUNAMAS', {'weight': 1.0}), ('CUNAMAS', 'PRONABEC', {'weight': 0.2}), ('CUNAMAS', 'QALIWARMA', {'weight': 0.33}), 

In [26]:
# comparissons for age as dict
inclusion_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_inclusion.edges(data=True) if e[0]!= e[1]}
inclusion_comparisons

{('PRONABEC', 'QALIWARMA'): 7.0,
 ('PRONABEC', 'JUNTOS'): 0.25,
 ('PRONABEC', 'FONCODES'): 3.0,
 ('PRONABEC', 'CUNAMAS'): 5.0,
 ('QALIWARMA', 'PRONABEC'): 0.14,
 ('QALIWARMA', 'JUNTOS'): 0.16,
 ('QALIWARMA', 'FONCODES'): 0.5,
 ('QALIWARMA', 'CUNAMAS'): 3.0,
 ('JUNTOS', 'PRONABEC'): 4.0,
 ('JUNTOS', 'QALIWARMA'): 6.0,
 ('JUNTOS', 'FONCODES'): 5.0,
 ('JUNTOS', 'CUNAMAS'): 7.0,
 ('FONCODES', 'PRONABEC'): 0.33,
 ('FONCODES', 'QALIWARMA'): 2.0,
 ('FONCODES', 'JUNTOS'): 0.2,
 ('FONCODES', 'CUNAMAS'): 5.0,
 ('CUNAMAS', 'PRONABEC'): 0.2,
 ('CUNAMAS', 'QALIWARMA'): 0.33,
 ('CUNAMAS', 'JUNTOS'): 0.14,
 ('CUNAMAS', 'FONCODES'): 0.2}

In [27]:
# the remaining comparissons:

G_benf = nx.from_pandas_adjacency(pairwise_beneficiarios,create_using=nx.MultiDiGraph())
beneficiarios_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_benf.edges(data=True) if e[0]!= e[1]}

G_premios = nx.from_pandas_adjacency(pairwise_premios,create_using=nx.MultiDiGraph())
premios_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_premios.edges(data=True) if e[0]!= e[1]}

In [28]:
# take a look
[inclusion_comparisons, beneficiarios_comparisons,premios_comparisons]

[{('PRONABEC', 'QALIWARMA'): 7.0,
  ('PRONABEC', 'JUNTOS'): 0.25,
  ('PRONABEC', 'FONCODES'): 3.0,
  ('PRONABEC', 'CUNAMAS'): 5.0,
  ('QALIWARMA', 'PRONABEC'): 0.14,
  ('QALIWARMA', 'JUNTOS'): 0.16,
  ('QALIWARMA', 'FONCODES'): 0.5,
  ('QALIWARMA', 'CUNAMAS'): 3.0,
  ('JUNTOS', 'PRONABEC'): 4.0,
  ('JUNTOS', 'QALIWARMA'): 6.0,
  ('JUNTOS', 'FONCODES'): 5.0,
  ('JUNTOS', 'CUNAMAS'): 7.0,
  ('FONCODES', 'PRONABEC'): 0.33,
  ('FONCODES', 'QALIWARMA'): 2.0,
  ('FONCODES', 'JUNTOS'): 0.2,
  ('FONCODES', 'CUNAMAS'): 5.0,
  ('CUNAMAS', 'PRONABEC'): 0.2,
  ('CUNAMAS', 'QALIWARMA'): 0.33,
  ('CUNAMAS', 'JUNTOS'): 0.14,
  ('CUNAMAS', 'FONCODES'): 0.2},
 {('PRONABEC', 'QALIWARMA'): 0.33,
  ('PRONABEC', 'JUNTOS'): 0.2,
  ('PRONABEC', 'FONCODES'): 3.0,
  ('PRONABEC', 'CUNAMAS'): 0.25,
  ('QALIWARMA', 'PRONABEC'): 3.0,
  ('QALIWARMA', 'JUNTOS'): 0.2,
  ('QALIWARMA', 'FONCODES'): 5.0,
  ('QALIWARMA', 'CUNAMAS'): 0.33,
  ('JUNTOS', 'PRONABEC'): 5.0,
  ('JUNTOS', 'QALIWARMA'): 5.0,
  ('JUNTOS', 'FONCOD

In [29]:
# now the criteria

G_CRIT = nx.from_pandas_adjacency(pairwise_criterios,create_using=nx.MultiDiGraph())
criterios_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_CRIT.edges(data=True) if e[0]!= e[1]}
criterios_comparisons

{('INCLUSION', 'BENEFICIARIOS'): 0.33,
 ('INCLUSION', 'RECONOCIMIENTO'): 0.2,
 ('BENEFICIARIOS', 'INCLUSION'): 0.2,
 ('BENEFICIARIOS', 'RECONOCIMIENTO'): 0.25,
 ('RECONOCIMIENTO', 'INCLUSION'): 5.0,
 ('RECONOCIMIENTO', 'BENEFICIARIOS'): 4.0}

### Paso 4: aplicar el algoritmo

In [30]:
!pip install numpy==1.24
!pip install ahpy

  Using cached numpy-1.24.0.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\maria\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\maria\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\maria\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "C:\Users\maria\AppData\Local\Programs\Python\Python312\Lib\si


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import ahpy

inclusion = ahpy.Compare('inclusión social', inclusion_comparisons, precision=3, random_index='saaty')
beneficiarios = ahpy.Compare('beneficiarios', beneficiarios_comparisons, precision=3, random_index='saaty')
premios = ahpy.Compare('premios', premios_comparisons, precision=3, random_index='saaty')
criterios = ahpy.Compare('criterios', criterios_comparisons, precision=3, random_index='saaty')

AttributeError: `itemset` was removed from the ndarray class in NumPy 2.0. Use `arr[index] = value` instead.

### Paso 5: Crear jerarquía

In [ ]:
criterios.add_children([inclusion, premios, beneficiarios])

### Paso 6: Visualizar los resultados

In [ ]:
print(criterios.target_weights)

### Paso 7: Evaluar la consistencia

In [ ]:
## We should review comparissons if greater than 0.1!
[(val.name,val.consistency_ratio) for val in [inclusion, premios, beneficiarios]]